<a href="https://colab.research.google.com/github/Dannatinjaca/PROGCOM-A/blob/main/PROGCOM_A_abstracci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from abc import ABC, abstractmethod

class PersonaBase(ABC):
    def __init__(self):
        self.puntos = 0

    @abstractmethod
    def evaluar(self):
        pass

class Persona(PersonaBase):
    def __init__(self):
        super().__init__()

    def evaluar(self):

        if input("¿Posee tarjetas con otras franquicias? (s/n): ").lower() == 's':
            self.puntos += 1

        if input("¿No está reportado ante entidades de riesgo? (s/n): ").lower() == 's':
            self.puntos += 3

        tipo_trabajo = input("¿Se encuentra laborando? (ninguno/termino_definido/termino_indefinido): ").lower()
        if tipo_trabajo == "termino_definido":
            self.puntos += 2
        elif tipo_trabajo == "termino_indefinido":
            self.puntos += 3

        if input("¿Había obtenido créditos en otra entidad bancaria? (s/n): ").lower() == 's':
            self.puntos += 2

        if input("¿Había obtenido crédito en la misma entidad bancaria? (s/n): ").lower() == 's':
            self.puntos += 3


        while True:
            edad_input = input("Ingrese la edad de la persona: ")
            if edad_input.isdigit():
                edad = int(edad_input)
                break
            else:
                print("Error: Debe ingresar un número válido para la edad.")

        if 18 <= edad <= 22:
            self.puntos += 2
        elif 23 <= edad <= 30:
            self.puntos += 4
        elif 31 <= edad <= 40:
            self.puntos += 3
        elif edad > 41:
            self.puntos += 1

        return self.puntos >= 6

def main():
    personas_con_credito = 0
    total_personas = int(input("Ingrese el número de personas a evaluar: "))

    for i in range(total_personas):
        print(f"\nEvaluando a la persona {i + 1}:")
        persona = Persona()
        if persona.evaluar():
            print(">> La persona puede obtener el crédito.")
            personas_con_credito += 1
        else:
            print(">> La persona NO puede obtener el crédito.")

    print(f"\nTotal de personas que obtuvieron el crédito: {personas_con_credito} de {total_personas}")

if __name__ == "__main__":
    main()


Ingrese el número de personas a evaluar: 2

Evaluando a la persona 1:
¿Posee tarjetas con otras franquicias? (s/n): s
¿No está reportado ante entidades de riesgo? (s/n): s
¿Se encuentra laborando? (ninguno/termino_definido/termino_indefinido): s
¿Había obtenido créditos en otra entidad bancaria? (s/n): s
¿Había obtenido crédito en la misma entidad bancaria? (s/n): s
Ingrese la edad de la persona: 18
>> La persona puede obtener el crédito.

Evaluando a la persona 2:
¿Posee tarjetas con otras franquicias? (s/n): s
¿No está reportado ante entidades de riesgo? (s/n): n
¿Se encuentra laborando? (ninguno/termino_definido/termino_indefinido): s
¿Había obtenido créditos en otra entidad bancaria? (s/n): n
¿Había obtenido crédito en la misma entidad bancaria? (s/n): a
Ingrese la edad de la persona: 18
>> La persona NO puede obtener el crédito.

Total de personas que obtuvieron el crédito: 1 de 2


In [7]:
!pip install gradio
import gradio as gr
from abc import ABC, abstractmethod


class PersonaBase(ABC):
    def __init__(self):
        self.puntos = 0

    @abstractmethod
    def evaluar(self, respuestas, edad):
        pass


class Persona(PersonaBase):
    def __init__(self):
        super().__init__()

    def evaluar(self, respuestas, edad):

        if respuestas['tarjeta'] == 'Sí':
            self.puntos += 1
        if respuestas['riesgo'] == 'Sí':
            self.puntos += 3
        if respuestas['trabajo'] == 'Término definido':
            self.puntos += 2
        elif respuestas['trabajo'] == 'Término indefinido':
            self.puntos += 3
        if respuestas['credito_otro'] == 'Sí':
            self.puntos += 2
        if respuestas['credito_mismo'] == 'Sí':
            self.puntos += 3


        if 18 <= edad <= 22:
            self.puntos += 2
        elif 23 <= edad <= 30:
            self.puntos += 4
        elif 31 <= edad <= 40:
            self.puntos += 3
        elif edad > 40:
            self.puntos += 1

        return self.puntos >= 6


def evaluar_credito(tarjeta, riesgo, trabajo, credito_otro, credito_mismo, edad_texto):
    try:
        edad = int(edad_texto)
        if edad < 18:
            return "❗ Debes tener al menos 18 años."
    except ValueError:
        return "⚠️ Error: La edad debe ser un número entero."

    respuestas = {
        "tarjeta": tarjeta,
        "riesgo": riesgo,
        "trabajo": trabajo,
        "credito_otro": credito_otro,
        "credito_mismo": credito_mismo
    }

    persona = Persona()
    aprobado = persona.evaluar(respuestas, edad)

    if aprobado:
        return "✅ Aprobado: Puede obtener el crédito."
    else:
        return "❌ Rechazado: No cumple con los requisitos del crédito."


with gr.Blocks() as interfaz:
    gr.Markdown("# 💳 Evaluación de Crédito Bancario")

    tarjeta = gr.Radio(["Sí", "No"], label="¿Tiene tarjetas con otras franquicias?")
    riesgo = gr.Radio(["Sí", "No"], label="¿NO está reportado ante entidades de riesgo?")
    trabajo = gr.Dropdown(["Ninguno", "Término definido", "Término indefinido"], label="Tipo de contrato")
    credito_otro = gr.Radio(["Sí", "No"], label="¿Ha obtenido créditos en otra entidad bancaria?")
    credito_mismo = gr.Radio(["Sí", "No"], label="¿Ha obtenido créditos en esta misma entidad?")
    edad = gr.Textbox(label="Edad (solo números)")

    boton = gr.Button("🔍 Evaluar Crédito")
    resultado = gr.Textbox(label="Resultado")

    boton.click(fn=evaluar_credito,
                inputs=[tarjeta, riesgo, trabajo, credito_otro, credito_mismo, edad],
                outputs=resultado)

interfaz.launch()
import gradio as gr
from abc import ABC, abstractmethod

# Clase abstracta base
class PersonaBase(ABC):
    def __init__(self):
        self.puntos = 0

    @abstractmethod
    def evaluar(self, respuestas, edad):
        pass


class Persona(PersonaBase):
    def __init__(self):
        super().__init__()

    def evaluar(self, respuestas, edad):

        if respuestas['tarjeta'] == 'Sí':
            self.puntos += 1
        if respuestas['riesgo'] == 'Sí':
            self.puntos += 3
        if respuestas['trabajo'] == 'Término definido':
            self.puntos += 2
        elif respuestas['trabajo'] == 'Término indefinido':
            self.puntos += 3
        if respuestas['credito_otro'] == 'Sí':
            self.puntos += 2
        if respuestas['credito_mismo'] == 'Sí':
            self.puntos += 3

        #Rango de edad

        if 18 <= edad <= 22:
            self.puntos += 2
        elif 23 <= edad <= 30:
            self.puntos += 4
        elif 31 <= edad <= 40:
            self.puntos += 3
        elif edad > 40:
            self.puntos += 1

        return self.puntos >= 6

# Función principal para evaluar crédito
def evaluar_credito(tarjeta, riesgo, trabajo, credito_otro, credito_mismo, edad_texto):
    try:
        edad = int(edad_texto)
        if edad < 18:
            return "❗ Debes tener al menos 18 años."
    except ValueError:
        return "⚠️ Error: La edad debe ser un número entero."

    respuestas = {
        "tarjeta": tarjeta,
        "riesgo": riesgo,
        "trabajo": trabajo,
        "credito_otro": credito_otro,
        "credito_mismo": credito_mismo
    }

    persona = Persona()
    aprobado = persona.evaluar(respuestas, edad)

    if aprobado:
        return "✅ Aprobado: Puede obtener el crédito."
    else:
        return "❌ Rechazado: No cumple con los requisitos del crédito."

# INTERFAZ GRÁFICA
with gr.Blocks() as interfaz:
    gr.Markdown("# 💳 Evaluación de Crédito Bancario")

    tarjeta = gr.Radio(["Sí", "No"], label="¿Tiene tarjetas con otras franquicias?")
    riesgo = gr.Radio(["Sí", "No"], label="¿NO está reportado ante entidades de riesgo?")
    trabajo = gr.Dropdown(["Ninguno", "Término definido", "Término indefinido"], label="Tipo de contrato")
    credito_otro = gr.Radio(["Sí", "No"], label="¿Ha obtenido créditos en otra entidad bancaria?")
    credito_mismo = gr.Radio(["Sí", "No"], label="¿Ha obtenido créditos en esta misma entidad?")
    edad = gr.Textbox(label="Edad (solo números)")

    boton = gr.Button("🔍 Evaluar Crédito")
    resultado = gr.Textbox(label="Resultado")

    boton.click(fn=evaluar_credito,
                inputs=[tarjeta, riesgo, trabajo, credito_otro, credito_mismo, edad],
                outputs=resultado)

interfaz.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://df86fe9dcdfe458bdb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://47043e3b34df7f9912.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
